In [34]:
import pandas as pd
from datetime import datetime, timedelta

# Adds 8 hours to timestamp (for GMT-8 tz) to make GMT time. Replace with your offset from GMT.
TZ_ADJ = 8

data_path = './data.csv'

In [35]:
def rsi(df, periods=14):
    """
    Returns a pd.Series with the relative strength index.
    From https://www.roelpeters.be/many-ways-to-calculate-the-rsi-in-python-pandas/
    """
    close_delta = df['close'].diff()

    # Make two series: one for lower closes and one for higher closes
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    
    # Use exponential moving average
    ma_up = up.ewm(com=periods-1, adjust=True, min_periods=periods).mean()
    ma_down = down.ewm(com=periods-1, adjust=True, min_periods=periods).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))
    return rsi

In [37]:
data = pd.read_csv(data_path)

# adjusting from GMT-8 local tz
data['time'] = data['time'].apply(lambda x: datetime.fromtimestamp(x) + timedelta(seconds=60*60*TZ_ADJ))

data['rsi'] = rsi(data)

In [38]:
# Find where RSI goes below 30 or above 70, and then retracts on the next bar. 
# Using the close of that bar as the entry, what is the MAE and MFE of the next 4 bars? 
